In [ ]:
import os
import sys
import logging
import cv2
import json
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from ast import literal_eval
from tqdm import tqdm_notebook as tqdm
import numpy as np

%matplotlib inline
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('TrainClassifier')

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from vehicle_detector import extract_features

In [ ]:
# Load the computer params and plot the results
with open('../data/params_stats.json') as fp:
    stats =  json.load(fp)['stats']

top_10_params = sorted(stats, key=lambda x: x['accuracy'])[:-11:-1]

accuracy = list(i ['accuracy'] for i in top_10_params)
extract_time = list(i ['extraction_time'] for i in top_10_params)
num_features = list(i ['num_features'] for i in top_10_params)
training_time = list(i ['training_time'] for i in top_10_params)
# top_10_params

In [ ]:
from sklearn.preprocessing import minmax_scale
width = 0.2

x = np.arange(len(accuracy))
plt.bar(x, minmax_scale(accuracy, feature_range=(0, 8)) + 2, width, label='Accuracy' )
plt.bar(x + width , minmax_scale(extract_time, feature_range=(0, 8)) + 2,width, label='Extract Time')
plt.bar(x + 2 * width , minmax_scale(num_features, feature_range=(0, 8)) + 2,width, label='Features Count')
plt.title('Parameter Combination & Performance')
plt.legend()
plt.show()

In [ ]:
# Load  Image Paths
with open('../data/filtered_images_path.json', 'r') as fp:
    data = json.load(fp)

vehicle_images_path = data['vehicle_images']
non_vehicle_images_path = data['non_vehicle_images']

In [ ]:
params_desc = map(lambda x: x['name'], top_10_params)

def get_dataset_name_from_params(params, hog_params):
    model_desc = 'A_{acc}_C_{c_val}_gamma_{gamma_val}'.format(acc=params['accuracy'],
                    c_val=params['best_params']['C'], gamma_val=params['best_params']['gamma'])
    param_desc= 'CH_{hist}_O_{orients}_P_{n_pixels}_C_{n_cells}_CS_{cspace}'.format(
        hist=hog_params['color_hist'], orients=hog_params['orientations'],
        n_pixels=hog_params['pixels_per_cell'], n_cells=hog_params['cells_per_block'],
        cspace=hog_params['hog_color_space'])
    
    return '{model}_{hog}.hdf5'.format(model=model_desc, hog=param_desc)

def get_feature_params(params_desc):
    tokens = params_desc.split('_')
    return {
        'hog_color_space': tokens[1],
        'pixels_per_cell': literal_eval(tokens[2][1:]),
        'cells_per_block': literal_eval(tokens[3][1:]),
        'orientations': literal_eval(tokens[4][1:]),
        'color_hist': literal_eval(tokens[6])
    }


In [ ]:
postive_samples = vehicle_images_path[:]
negative_samples = non_vehicle_images_path[:]

dataset_dir = '../data/datasets'

if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
        
for params in tqdm(top_10_params, desc='Creating Datasets'):
    feature_params = get_feature_params(params['name'])
    dataset_name = get_dataset_name_from_params(params, feature_params)
    dataset_path = os.path.join(dataset_dir, dataset_name)
    extract_features.extract_save_features(postive_samples, negative_samples, feature_params,
                                          dataset_path, 'carnd_p5')

